# Quickstart Guide
This guide is adapted from Cocotb's [README](https://github.com/cocotb/cocotb#usage)

In [ ]:
import random

import cocotb
from cocotb.clock import Clock
from cocotb.triggers import RisingEdge
from cocotb.task import resume
from cocotbext.waves import waveform

import wavedrom
import ipywidgets as widgets
from ipywidgets import AppLayout, VBox, Layout

### Setup the clock

Create a 10us period clock and connect it to the DUT's clk port.



In [ ]:
clock = Clock(dut.clk, 10, unit="us")
cocotb.start_soon(clock.start(start_high=False));

### Setup the waveform monitor
Waveform support is provided by the `cocotbext-waves` package since wavedrom support was remove in cocotb 2.0

The monitor will capture a waveform from the D flip-flop

In [ ]:
waves = waveform(clk=dut.clk, name="dff", start=True)
waves.add_signal(
    [
        dut.d, 
        dut.q
    ]
)                                    

### Test the D flip-flop with random values

Code within a cell is ran within a cocotb "test", so `await` works without needing to declare an `async` function. Future verisons may add the ability to declare "test" functions that will be passed to cocotb's `RegressionManager`.

In [ ]:
await RisingEdge(dut.clk)
expected_val = 0
for i in range(10):
    val = random.randint(0, 1)
    dut.d.value = val
    await RisingEdge(dut.clk)
    assert dut.q.value == expected_val, f"output q was incorrect on the {i}th cycle"
    expected_val = val

# Use wavedrom.render instead of save_svg
display(wavedrom.render(str(waves)))

### Toggle D flip-flop

Interactivity can be added by using `ipywidgets`. In the code below, an async function, `step_random()` is called whenver the *Step* button is pressed. 

For asynchronous callbacks, be sure to wrap the callback with the `resume()` function

In [ ]:
out = widgets.Output(layout=Layout(overflow='scroll'))

async def step(b):
    await RisingEdge(dut.clk)
    out.outputs = ()
    out.append_display_data(wavedrom.render(str(waves)))

step_button = widgets.Button(description="Step")
step_button.on_click(resume(step))

def toggle_dff(change):
    new_value = int(change["new"])
    toggle_button.description = f'd={str(new_value)}'
    dut.d.value = new_value
    
toggle_button = widgets.ToggleButton(value=bool(dut.d.value), description=f'd={dut.d.value}')
toggle_button.observe(toggle_dff, names="value")

AppLayout(header = None,
          left_sidebar = VBox([step_button,
                               toggle_button],
                              layout=Layout(flex='0 0 auto')
                             ),
          center = out,
          right_sidebar = None,
          footer = None,
          height='100px',
          layout=Layout(display='flex',
                        overflow='hidden',
                        justify_content='flex-start')
          )
          